In [5]:
from collections import Counter, defaultdict
import os
import  regex as re
from concurrent.futures import ThreadPoolExecutor

def split_on_special(text, special_tokens):
    """
    Splits text into (is_special, chunk) pairs, where is_special indicates if chunk is a special token.
    Ensures special tokens are treated atomically.
    """
    # Sort by length descending to avoid partial matches
    pattern = re.compile("|".join(re.escape(tok) for tok in sorted(special_tokens, key=len, reverse=True)))
    chunks = []
    last = 0
    for m in pattern.finditer(text):
        if m.start() > last:
            chunks.append((False, text[last:m.start()]))
        chunks.append((True, m.group()))
        last = m.end()
    if last < len(text):
        chunks.append((False, text[last:]))
    return chunks

def pretokenize(text):
    PAT = r"""'(?:[sdmt]|ll|ve|re)| ?\p{L}+| ?\p{N}+| ?[^\s\p{L}\p{N}]+|\s+(?!\S)|\s+"""
    pattern = re.compile(PAT)
    return [m.group(0) for m in pattern.finditer(text)]

def process_chunk(chunk, special_tokens):
    # Split on special tokens, then pretokenize each part
    pre_token_counts = Counter()
    pattern = re.compile("|".join(re.escape(tok) for tok in sorted(special_tokens, key=len, reverse=True)))
    chunks = pattern.split(chunk)
    # specials = special_pattern.findall(chunk)
    for i, chunk_text in enumerate(chunks):
        for tok in pretokenize(chunk_text):
            pre_token_counts[tok.encode('utf-8')] += 1

    return pre_token_counts
def process_chunk_args(args):
    chunk, special_tokens = args
    return process_chunk(chunk, special_tokens)


def run_train_bpe(
    input_path: str | os.PathLike,
    vocab_size: int,
    special_tokens: list[str],
    **kwargs,
) -> tuple[dict[int, bytes], list[tuple[bytes, bytes]]]:
    """
    Train a BPE tokenizer matching GPT-2's behavior.
    Returns:
        vocab: Mapping from token ID to bytes.
        merges: List of merge pairs in order of creation.
    """
    # Initialize vocabulary with single bytes
    vocab: dict[int, bytes] = {i: bytes([i]) for i in range(256)}
    token_to_id: dict[bytes, int] = {v: k for k, v in vocab.items()}
    special_token_bytes = [s.encode("utf-8") for s in special_tokens]
    merges: list[tuple[bytes, bytes]] = []
    current_id = 256

    num_processes = 4

    with open(input_path, "rb") as f:
        boundaries = find_chunk_boundaries(
            f, num_processes, "<|endoftext|>".encode("utf-8"))
        chunks = []
        for start, end in zip(boundaries[:-1], boundaries[1:]):
            f.seek(start)
            chunk = f.read(end - start).decode("utf-8", errors="ignore")
            chunks.append(chunk)

    # Prepare the argument list
    args_list = [(chunk, special_tokens) for chunk in chunks]

    with ThreadPoolExecutor(max_workers=num_processes) as executor:
        counters = list(executor.map(process_chunk_args, args_list))

    from functools import reduce
    from operator import add
    pre_token_counts = reduce(add, counters, Counter())

    # Build corpus: list of (tuple of token ids, freq)
    # **Ensures special tokens are always atomic**
    for token in special_token_bytes:
        if token not in vocab.values():
            vocab[current_id] = token
            token_to_id[token] = current_id
            current_id += 1
    token_to_id = {v: k for k, v in vocab.items()}  # Rebuild for safety

    corpus = []
    for pretoken_bytes, freq in pre_token_counts.items():

        tok_ids = tuple(token_to_id[bytes([b])] for b in pretoken_bytes)
        corpus.append((tok_ids, freq))

    # special_token_ids = set(token_to_id[tok] for tok in special_token_bytes)

    # BPE merge loop
    while current_id < vocab_size:
        pair_freq = Counter()
        pair_to_position = defaultdict(set)
        for idx, (token_seq, freq) in enumerate(corpus):
            for i in range(len(token_seq) - 1):
                a, b = token_seq[i], token_seq[i + 1]
                # Never merge across special tokens!

                pair = (a, b)
                pair_freq[pair] += freq
                pair_to_position[pair].add((idx,i))
                # if pair not in pair_first_pos:
                #     pair_first_pos[pair] = idx

        if not pair_freq:
            break

        max_count = max(pair_freq.values())
        candidates = [pair for pair, count in pair_freq.items() if count == max_count]

        # Select the lexicographically greatest pair
        best_pair = max(candidates)

        merged_bytes = vocab[best_pair[0]] + vocab[best_pair[1]]

        vocab[current_id] = merged_bytes
        token_to_id[merged_bytes] = current_id
        merges.append((vocab[best_pair[0]], vocab[best_pair[1]]))

        # Merge pairs in the corpus

        # For each position where best_pair appears
        positions = list(pair_to_position[best_pair])
        for seq_idx, i in positions:
            token_seq, freq = corpus[seq_idx]
            token_seq = list(token_seq)
            # Skip if already merged (could happen if two merges affect same place)
            if i >= len(token_seq) - 1:
                continue
            if (token_seq[i], token_seq[i + 1]) != best_pair:
                continue

            left = token_seq[i - 1] if i - 1 >= 0 else None
            right = token_seq[i + 2] if i + 2 < len(token_seq) else None

            # Remove freq from left and right pairs (old)
            if left is not None:
                old_left = (left, token_seq[i])
                pair_freq[old_left] -= freq
                pair_to_position[old_left].discard((seq_idx, i - 1))
            if right is not None:
                old_right = (token_seq[i + 1], right)
                pair_freq[old_right] -= freq
                pair_to_position[old_right].discard((seq_idx, i + 1))

            # Remove freq from the best_pair
            pair_freq[best_pair] -= freq
            pair_to_position[best_pair].discard((seq_idx, i))

            # Merge best_pair -> new_token
            token_seq[i:i + 2] = [current_id]

            # Add new pairs (left and right of new_token)
            if left is not None:
                new_left = (left, current_id)
                pair_freq[new_left] += freq
                pair_to_position[new_left].add((seq_idx, i - 1))
            if right is not None:
                new_right = (current_id, right)
                pair_freq[new_right] += freq
                pair_to_position[new_right].add((seq_idx, i))

            # Save updated sequence back to corpus
            corpus[seq_idx] = (tuple(token_seq), freq)

        # Clean up: remove pairs with zero freq or empty positions
        # pair_freq = Counter({p: c for p, c in pair_freq.items() if c > 0})
        # pair_to_positions = {p: s for p, s in pair_to_position.items() if s}



        current_id += 1



    return vocab, merges


import os
from typing import BinaryIO


def find_chunk_boundaries(
        file: BinaryIO,
        desired_num_chunks: int,
        split_special_token: bytes
) -> list[int]:
    """
    Chunk the file into parts that can be counted independently.
    May return fewer chunks if the boundaries end up overlapping.
    """
    assert isinstance(split_special_token, bytes), (
        "Must represent special token as a bytestring"
    )

    # Get total file size in bytes
    file.seek(0, os.SEEK_END)
    file_size = file.tell()
    file.seek(0)

    chunk_size = file_size // desired_num_chunks

    # Initial guesses for chunk boundary locations, uniformly spaced
    # Chunks start on previous index, don't include last index
    chunk_boundaries = [i * chunk_size for i in range(desired_num_chunks + 1)]
    chunk_boundaries[-1] = file_size

    mini_chunk_size = 4096  # Read ahead by 4k bytes at a time

    for bi in range(1, len(chunk_boundaries) - 1):
        initial_position = chunk_boundaries[bi]
        file.seek(initial_position)  # Start at boundary guess
        while True:
            mini_chunk = file.read(mini_chunk_size)  # Read a mini chunk

            # If EOF, this boundary should be at the end of the file
            if mini_chunk == b"":
                chunk_boundaries[bi] = file_size
                break

            # Find the special token in the mini chunk
            found_at = mini_chunk.find(split_special_token)
            if found_at != -1:
                chunk_boundaries[bi] = initial_position + found_at
                break
            initial_position += mini_chunk_size

    # Make sure all boundaries are unique, but might be fewer than desired_num_chunks
    return sorted(set(chunk_boundaries))


In [6]:
run_train_bpe("tinystories_sample.txt",1000,["<|endoftext|>"])

({0: b'\x00',
  1: b'\x01',
  2: b'\x02',
  3: b'\x03',
  4: b'\x04',
  5: b'\x05',
  6: b'\x06',
  7: b'\x07',
  8: b'\x08',
  9: b'\t',
  10: b'\n',
  11: b'\x0b',
  12: b'\x0c',
  13: b'\r',
  14: b'\x0e',
  15: b'\x0f',
  16: b'\x10',
  17: b'\x11',
  18: b'\x12',
  19: b'\x13',
  20: b'\x14',
  21: b'\x15',
  22: b'\x16',
  23: b'\x17',
  24: b'\x18',
  25: b'\x19',
  26: b'\x1a',
  27: b'\x1b',
  28: b'\x1c',
  29: b'\x1d',
  30: b'\x1e',
  31: b'\x1f',
  32: b' ',
  33: b'!',
  34: b'"',
  35: b'#',
  36: b'$',
  37: b'%',
  38: b'&',
  39: b"'",
  40: b'(',
  41: b')',
  42: b'*',
  43: b'+',
  44: b',',
  45: b'-',
  46: b'.',
  47: b'/',
  48: b'0',
  49: b'1',
  50: b'2',
  51: b'3',
  52: b'4',
  53: b'5',
  54: b'6',
  55: b'7',
  56: b'8',
  57: b'9',
  58: b':',
  59: b';',
  60: b'<',
  61: b'=',
  62: b'>',
  63: b'?',
  64: b'@',
  65: b'A',
  66: b'B',
  67: b'C',
  68: b'D',
  69: b'E',
  70: b'F',
  71: b'G',
  72: b'H',
  73: b'I',
  74: b'J',
  75: b'K',
  76: b'

In [2]:
"""low low low low low
lower lower widest widest widest
newest newest newest newest newest newest
and the vocabulary has a special token <|endoftext|>.""".split("<|endoftext|>")

['low low low low low\nlower lower widest widest widest\nnewest newest newest newest newest newest\nand the vocabulary has a special token ',
 '.']

In [30]:
text="""u don't have to be scared of the loud dog, I'll protect you". The mole felt so safe with the little girl. She was very kind and the mole soon came to trust her. He leaned against her and she kept him safe. The mole had found his best friend.
<|endoftext|>
Once upon a time, in a warm and sunny place, there was a big pit. A little boy named Tom liked to play near the pit. One day, Tom lost his red ball. He was very sad.
Tom asked his friend, Sam, to help him search for the ball. They looked high and low, but they could not find the ball. Tom said, "I think my ball fell into the pit."
Sam and Tom went close to the pit. They were scared, but they wanted to find the red ball. They looked into the pit, but it was too dark to see. Tom said, "We must go in and search for my ball."
They went into the pit to search. It was dark and scary. They could not find the ball. They tried to get out, but the pit was too deep. Tom and Sam were stuck in the pit. They called for help, but no one could hear them. They were sad and scared, and they never got out of the pit.
<|endoftext|>


Tom and Lily were playing with their toys in the living room. They liked to build towers and bridges with their blocks and cars. Tom was very proud of his tall tower. He wanted to make it even taller, so he reached for more blocks.
"Tom, can I have some blocks too?" Lily asked. She wanted to make a bridge for her cars.
"No, these are mine. Go find your own," Tom said. He did not want to share with his sister. He pulled the blocks closer to him.
Lily felt sad and angry. She did not think Tom was being nice. She looked at his tower and had an idea. She decided to pull one of the blocks at the bottom of the tower.
Suddenly, the tower fell down with a loud crash. All the blocks and cars scattered on the floor. Tom and Lily were shocked. They felt the floor shake and heard a rumble. It was an earthquake!
"Mommy! Daddy!" they cried. They were scared and ran to their parents, who were in the kitchen.
"Are you okay, kids?" Mommy asked. She hugged them and checked if they were hurt.
"We're okay, Mommy. But our toys are broken," Lily said.
"I'm sorry, Lily. But toys are not important. You are important. We are safe and together. That's what matters," Mommy said.
Tom felt sorry for what he did. He realized he was selfish and mean to his sister. He saw how scared she was during the earthquake. He wanted to make her happy.
"Lily, I'm sorry I did not share with you. You can have all the blocks you want. I love you, sister," Tom said.
Lily smiled and hugged him. She forgave him and thanked him. She loved him too.
They went back to the living room and cleaned up their toys. They decided to build something together. They made a big house with a garden and a fence. They put their cars and dolls inside. They were happy and proud of their work.
Mommy and Daddy came to see their house. They praised them and gave them a treat. It was a lemon cake. It was sour, but they liked it. They learned that sharing is caring, and that family is sweet.
<|endoftext|>

Once upon a time there was a little girl named Lucy. She loved to go to the store to buy sweets with her mom and dad. On this special day, Lucy entered the store with her mom and dad, feeling so excited.
As they were looking around, Lucy noticed a little girl playing with a toy in the corner of the store. She gasped in excitement and ran towards her. Lucy asked if she could play too but the little girl said no. She was rather grumpy and was not in the mood to play.
Lucy's mom saw what was going on and told Lucy, "Let's try to be peaceful and kind to her. Have patience and understanding. Together, you can both be happy!"
So, Lucy smiled at the girl and said, "Can we play together?" The little girl softened and smiled back. She agreed to share the toy and even let Lucy have a turn first.
Lucy and the little girl played together happily. In the end, they both learnt an important lesson: be peaceful, kind, and understanding when faced with a conflict. And that is why Lucy and the little girl became great friends.
<|endoftext|>"""

In [19]:
def pretokenize(text):
    PAT = r"""'(?:[sdmt]|ll|ve|re)| ?\p{L}+| ?\p{N}+| ?[^\s\p{L}\p{N}]+|\s+(?!\S)|\s+"""
    pattern = re.compile(PAT)
    return [m.group(0) for m in pattern.finditer(text)]


In [32]:
def process_chunk(chunk, special_tokens):
    # Split on special tokens, then pretokenize each part
    pre_token_counts = Counter()
    pattern = re.compile("|".join(re.escape(tok) for tok in sorted(special_tokens, key=len, reverse=True)))
    print(pattern)
    chunks = pattern.split(chunk)
    # specials = special_pattern.findall(chunk)
    print(chunks)
    for i, chunk_text in enumerate(chunks):
        for tok in pretokenize(chunk_text):
            pre_token_counts[tok.encode('utf-8')] += 1

    return pre_token_counts

In [33]:
# pretokenize(text)

In [34]:
text="""0 And inasmuch as they are a faithful they shall be preserved , and I , the Lord , will be b with them .
11 And let the residue take that which is needful for clothing .
12 Let my servant Sidney Gilbert take that which is not needful with him , as you shall agree .
13 And now , behold , for your a good I gave unto you a b commandment concerning these things ; and I , the Lord , will reason with you as with men in days of old .
14 Behold , I , the Lord , in the beginning blessed the a waters ; but in the last days , by the mouth of my servant John , I b cursed the waters .
15 Wherefore , the days will come that no flesh shall be safe upon the waters .
16 And it shall be said in days to come that none is able to go up to the land of Zion upon the waters , but he that is upright in heart .
17 And , as I , the Lord , in the beginning a cursed the land , even so in the last days have I b b"""

In [35]:
process_chunk(text,["<|endoftext|>","eot"])

regex.Regex('<\\|endoftext\\|>|eot', flags=regex.V0)
['0 And inasmuch as they are a faithful they shall be preserved , and I , the Lord , will be b with them .\n11 And let the residue take that which is needful for clothing .\n12 Let my servant Sidney Gilbert take that which is not needful with him , as you shall agree .\n13 And now , behold , for your a good I gave unto you a b commandment concerning these things ; and I , the Lord , will reason with you as with men in days of old .\n14 Behold , I , the Lord , in the beginning blessed the a waters ; but in the last days , by the mouth of my servant John , I b cursed the waters .\n15 Wherefore , the days will come that no flesh shall be safe upon the waters .\n16 And it shall be said in days to come that none is able to go up to the land of Zion upon the waters , but he that is upright in heart .\n17 And , as I , the Lord , in the beginning a cursed the land , even so in the last days have I b b']


Counter({b' ,': 19,
         b' the': 17,
         b' I': 7,
         b' .': 7,
         b'\n': 7,
         b' in': 7,
         b' And': 5,
         b' a': 5,
         b' b': 5,
         b' that': 5,
         b' days': 5,
         b' as': 4,
         b' shall': 4,
         b' be': 4,
         b' Lord': 4,
         b' with': 4,
         b' is': 4,
         b' waters': 4,
         b' will': 3,
         b' you': 3,
         b' of': 3,
         b' to': 3,
         b' they': 2,
         b' and': 2,
         b' take': 2,
         b' which': 2,
         b' needful': 2,
         b' for': 2,
         b' my': 2,
         b' servant': 2,
         b' ;': 2,
         b' beginning': 2,
         b' but': 2,
         b' last': 2,
         b' cursed': 2,
         b' come': 2,
         b' upon': 2,
         b' land': 2,
         b'0': 1,
         b' inasmuch': 1,
         b' are': 1,
         b' faithful': 1,
         b' preserved': 1,
         b' them': 1,
         b'11': 1,
         b' let': 1,
      

In [26]:
from collections import Counter, defaultdict
from heapq import heapify, heappop
from functools import partial, reduce
from concurrent.futures import ProcessPoolExecutor
from operator import add
import os
import regex as re
from typing import BinaryIO

def gpt2_bytes_to_unicode():
    bs = (
        list(range(ord("!"), ord("~") + 1))
        + list(range(ord("¡"), ord("¬") + 1))
        + list(range(ord("®"), ord("ÿ") + 1))
    )
    cs = bs[:]
    n = 0
    for b in range(256):
        if b not in bs:
            bs.append(b)
            cs.append(256 + n)
            n += 1
    return dict(zip(bs, [chr(c) for c in cs]))

def pretokenize(text):
    PAT = r"""'(?:[sdmt]|ll|ve|re)| ?\p{L}+| ?\p{N}+| ?[^\s\p{L}\p{N}]+|\s+(?!\S)|\s+"""
    pattern = re.compile(PAT)
    return [m.group(0) for m in pattern.finditer(text)]

def split_on_special(text, special_tokens):
    pattern = re.compile("|".join(re.escape(tok) for tok in sorted(special_tokens, key=len, reverse=True)))
    chunks = []
    last = 0
    for m in pattern.finditer(text):
        if m.start() > last:
            chunks.append((False, text[last:m.start()]))
        chunks.append((True, m.group()))
        last = m.end()
    if last < len(text):
        chunks.append((False, text[last:]))
    return chunks

def process_chunk(chunk, special_tokens):
    pre_token_counts = Counter()
    for is_special, chunk_text in split_on_special(chunk, special_tokens):
        if is_special:
            pre_token_counts[chunk_text.encode('utf-8')] += 1
        else:
            for tok in pretokenize(chunk_text):
                try:
                    pre_token_counts[tok.encode('utf-8')] += 1
                except UnicodeEncodeError:
                    pre_token_counts[tok.encode('latin-1')] += 1
    return pre_token_counts

def process_chunk_args(args):
    chunk, special_tokens = args
    return process_chunk(chunk, special_tokens)

def find_chunk_boundaries(
    file: BinaryIO,
    desired_num_chunks: int,
    split_special_token: bytes
) -> list[int]:
    file.seek(0, os.SEEK_END)
    file_size = file.tell()
    file.seek(0)
    chunk_size = file_size // desired_num_chunks
    chunk_boundaries = [i * chunk_size for i in range(desired_num_chunks + 1)]
    chunk_boundaries[-1] = file_size
    mini_chunk_size = 4096
    for bi in range(1, len(chunk_boundaries) - 1):
        initial_position = chunk_boundaries[bi]
        file.seek(initial_position)
        while True:
            mini_chunk = file.read(mini_chunk_size)
            if not mini_chunk:
                chunk_boundaries[bi] = file_size
                break
            found_at = mini_chunk.find(split_special_token)
            if found_at != -1:
                chunk_boundaries[bi] = initial_position + found_at + len(split_special_token)
                break
            initial_position += mini_chunk_size
    return sorted(set(chunk_boundaries))

def run_train_bpe(
    input_path: str | os.PathLike,
    vocab_size: int,
    special_tokens: list[str],
    **kwargs,
) -> tuple[dict[int, bytes], list[tuple[bytes, bytes]]]:
    vocab: dict[int, bytes] = {i: bytes([i]) for i in range(256)}
    token_to_id: dict[bytes, int] = {v: k for k, v in vocab.items()}
    special_token_bytes = [s.encode("utf-8") for s in special_tokens]
    merges: list[tuple[bytes, bytes]] = []
    current_id = 256
    for token in special_token_bytes:
        if token not in token_to_id:
            vocab[current_id] = token
            token_to_id[token] = current_id
            current_id += 1
    print(f"token_to_id contains b'<|endoftext|>': {b'<|endoftext|>' in token_to_id}")

    num_processes = 1
    with open(input_path, "rb") as f:
        boundaries = find_chunk_boundaries(
            f, num_processes, "<|endoftext|>".encode("utf-8"))
        chunks = []
        for start, end in zip(boundaries[:-1], boundaries[1:]):
            f.seek(start)
            chunk = f.read(end - start).decode("utf-8", errors="ignore")
            chunks.append(chunk)

    args_list = [(chunk, special_tokens) for chunk in chunks]
    with ThreadPoolExecutor(max_workers=num_processes) as executor:
        counters = list(executor.map(process_chunk_args, args_list))
    pre_token_counts = reduce(add, counters, Counter())

    print(f"Top pretokens: {pre_token_counts.most_common(10)}")

    corpus = []
    special_token_set = set(special_token_bytes)
    for pretoken_bytes, freq in pre_token_counts.items():
        if pretoken_bytes in special_token_set:
            tok_id = token_to_id[pretoken_bytes]
            corpus.append(([tok_id], freq))
        else:
            try:
                text = pretoken_bytes.decode('utf-8')
            except UnicodeDecodeError:
                text = pretoken_bytes.decode('latin-1')
            tok_ids = []
            for char in text:
                char_bytes = char.encode('utf-8')
                if char_bytes in token_to_id:
                    tok_ids.append(token_to_id[char_bytes])
                else:
                    tok_ids.extend(token_to_id[bytes([b])] for b in char_bytes)
            corpus.append((tok_ids, freq))
    print(f"Sample corpus: {corpus[:5]}")

    special_token_set = set(token_to_id[tok] for tok in special_token_bytes)
    while current_id < vocab_size:
        pair_freq = Counter()
        pair_to_position = defaultdict(set)
        for idx, (token_seq, freq) in enumerate(corpus):
            for i in range(len(token_seq) - 1):
                a, b = token_seq[i], token_seq[i + 1]
                if a in special_token_set or b in special_token_set:
                    continue
                pair = (a, b)
                pair_freq[pair] += freq
                pair_to_position[pair].add((idx, i))

        if not pair_freq:
            break

        

        heap = [(-freq, min(pair_to_position[pair])[0], pair) for pair in pair_freq]
        heapify(heap)
        best_pair = heappop(heap)[-1]
        merged_bytes = vocab[best_pair[0]] + vocab[best_pair[1]]
        vocab[current_id] = merged_bytes
        token_to_id[merged_bytes] = current_id
        merges.append((vocab[best_pair[0]], vocab[best_pair[1]]))
        if current_id <= 271:
            print(f"Merge {current_id - 256}: {vocab[best_pair[0]]} {vocab[best_pair[1]]}")
        if current_id == 271:
            le_id = (token_to_id[b'l'], token_to_id[b'e'])
            ce_id = (token_to_id[b'c'], token_to_id[b'e'])
            print(f"Pair counts: {pair_freq.most_common(10)}")
            print(f"(b'l', b'e') count: {pair_freq.get(le_id, 0)}")
            print(f"(b'c', b'e') count: {pair_freq.get(ce_id, 0)}")
            print(f"Chosen pair: {vocab[best_pair[0]]} {vocab[best_pair[1]]}")
        new_corpus = []
        for token_seq, freq in corpus:
            merged = []
            i = 0
            while i < len(token_seq):
                if i < len(token_seq) - 1 and (token_seq[i], token_seq[i + 1]) == best_pair:
                    merged.append(current_id)
                    i += 2
                else:
                    merged.append(token_seq[i])
                    i += 1
            new_corpus.append((merged, freq))
        corpus = new_corpus
        current_id += 1

    return vocab, merges

In [27]:
run_train_bpe("./fixtures/corpus.en",1000,["<|endoftext|>"])

token_to_id contains b'<|endoftext|>': True
Top pretokens: [(b' ,', 1306), (b' the', 1279), (b'\n', 1015), (b' .', 962), (b' of', 646), (b' and', 609), (b' a', 480), (b' to', 474), (b' in', 436), (b';', 357)]
Sample corpus: [([105, 114, 111, 110], 2), ([32, 99, 101, 109, 101, 110, 116], 3), ([32, 105, 115], 338), ([32, 97], 480), ([32, 114, 101, 97, 100, 121], 4)]
Merge 1: b'i' b'r'
Merge 2: b'o' b'n'
Merge 3: b'ir' b'on'
Merge 4: b' ' b'c'
Merge 5: b'e' b'm'
Merge 6: b'e' b'n'
Merge 7: b' c' b'em'
Merge 8: b'en' b't'
Merge 9: b' cem' b'ent'
Merge 10: b' ' b'i'
Merge 11: b' i' b's'
Merge 12: b' ' b'a'
Merge 13: b' ' b'r'
Merge 14: b'a' b'd'
Merge 15: b'e' b'ad'
Merge 16: b' r' b'ead'
Merge 17: b' read' b'y'
Merge 18: b' ' b'f'
Merge 19: b'o' b'r'
Merge 20: b' f' b'or'
Merge 21: b' ' b'u'
Merge 22: b's' b'e'
Merge 23: b' u' b'se'
Merge 24: b' ' b'p'
Merge 25: b'a' b's'
Merge 26: b't' b'e'
Merge 27: b' p' b'as'
Merge 28: b' pas' b'te'
Merge 29: b' ' b'w'
Merge 30: b'c' b'h'
Merge 31: b'h

({0: b'\x00',
  1: b'\x01',
  2: b'\x02',
  3: b'\x03',
  4: b'\x04',
  5: b'\x05',
  6: b'\x06',
  7: b'\x07',
  8: b'\x08',
  9: b'\t',
  10: b'\n',
  11: b'\x0b',
  12: b'\x0c',
  13: b'\r',
  14: b'\x0e',
  15: b'\x0f',
  16: b'\x10',
  17: b'\x11',
  18: b'\x12',
  19: b'\x13',
  20: b'\x14',
  21: b'\x15',
  22: b'\x16',
  23: b'\x17',
  24: b'\x18',
  25: b'\x19',
  26: b'\x1a',
  27: b'\x1b',
  28: b'\x1c',
  29: b'\x1d',
  30: b'\x1e',
  31: b'\x1f',
  32: b' ',
  33: b'!',
  34: b'"',
  35: b'#',
  36: b'$',
  37: b'%',
  38: b'&',
  39: b"'",
  40: b'(',
  41: b')',
  42: b'*',
  43: b'+',
  44: b',',
  45: b'-',
  46: b'.',
  47: b'/',
  48: b'0',
  49: b'1',
  50: b'2',
  51: b'3',
  52: b'4',
  53: b'5',
  54: b'6',
  55: b'7',
  56: b'8',
  57: b'9',
  58: b':',
  59: b';',
  60: b'<',
  61: b'=',
  62: b'>',
  63: b'?',
  64: b'@',
  65: b'A',
  66: b'B',
  67: b'C',
  68: b'D',
  69: b'E',
  70: b'F',
  71: b'G',
  72: b'H',
  73: b'I',
  74: b'J',
  75: b'K',
  76: b'